In [2]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Ridge
import numpy as np
import sklearn.datasets
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsRegressor
from sklearn import ensemble
import time
import pickle
import random
from sklearn.preprocessing import OneHotEncoder
from sklearn import cross_validation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
import tensorflow as tf
import tensorflow as tf
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score , recall_score , f1_score

In [3]:
train=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/train.csv')
test=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/test.csv')
sample_submission=pd.read_csv('/Users/s0c02nj/Desktop/c3cc8568-0-dataset/sample_submission.csv')

In [4]:
train_copy=train.copy()

#Test

In [5]:
#Joining Available Columns in Test and Train for Pre-processing
join=train_copy.append(test,sort=False)


In [6]:
#Date Time
join['Date-received'] =  pd.to_datetime(join['Date-received'], format='%m/%d/%Y')
join['Date-sent-to-company'] =  pd.to_datetime(join['Date-sent-to-company'], format='%m/%d/%Y')

In [7]:
#Adding Time Related Columns

# join['hour_recvd']= join['Date-received'].apply(lambda x: correct_spelling(x, x.hours))
# join['month_recvd']=join['Date-received'].apply(lambda x: correct_spelling(x, x.months))
# join['year_recvd']= join['Date-received'].apply(lambda x: correct_spelling(x, x.years))


In [8]:
join['diff_time']=join['Date-sent-to-company']-join['Date-received']
join['diff_time']=join['diff_time'].apply(lambda x:x.days )

Making a Separate category for missing

In [9]:
#Company Respose
join['Company-response'].fillna(value='No response',inplace =True)

#Missing Dispute-- No Dispute
join['Consumer-disputes'].fillna(value=join['Consumer-disputes'].value_counts().index[0],inplace =True)

In [10]:
count_trans=len(join['Transaction-Type'].unique())
print count_trans

18


In [11]:
count_complain_reason=len(join['Complaint-reason'].unique())
print count_complain_reason

152


In [12]:
count_company_response=len(join['Company-response'].unique())
print count_company_response

11


In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [14]:
diff_time=np.array(join['diff_time']).reshape(-1,1)

In [15]:
enc_time=scaler.fit_transform(diff_time)

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Processing The categorical Variables

In [16]:
#1.Label Encoding the Categorical Varaible----TransactionType
le1= preprocessing.LabelEncoder()
enc_trans_type=le1.fit_transform(join['Transaction-Type'])

In [17]:
#2.Label Encoding the Categorical Varaible----Complaint-reason
le2= preprocessing.LabelEncoder()
enc_complain_reason=le2.fit_transform(join['Complaint-reason'])

In [18]:
#3.Label Encoding the Categorical Varaible----Company-response
le3= preprocessing.LabelEncoder()
enc_comp_response=le3.fit_transform(join['Company-response'])

In [19]:
#4.Label Encoding the Categorical Varaible----Consumer-disputes
le4= preprocessing.LabelEncoder()
enc_cons_dispute=le4.fit_transform(join['Consumer-disputes'])

In [20]:
#Response 
le5= preprocessing.LabelEncoder()
y_cat=le5.fit_transform(train['Complaint-Status'])

In [21]:
from keras.utils import to_categorical
y_enc=to_categorical(y_cat,num_classes=5)

Using TensorFlow backend.


Processing the Text Data Column----> Consumer-complaint-summary

In [22]:
join['Consumer-complaint-summary'] = join['Consumer-complaint-summary'].apply(lambda x: re.sub(r'[^a-zA-Z]', ' ', x))

In [23]:
join['Consumer-complaint-summary'] = join['Consumer-complaint-summary'].apply(lambda x: str(x))

In [24]:
join['Consumer-complaint-summary']=join['Consumer-complaint-summary'].str.replace('XX','')

In [25]:
join['Consumer-complaint-summary']=join['Consumer-complaint-summary'].apply(lambda x:x.lower())

Featue Engineering on Text

In [26]:
data_len = join['Consumer-complaint-summary'].apply(lambda x: len(str(x)))
data_len_char = join['Consumer-complaint-summary'].apply(lambda x: len(''.join(set(str(x).replace(' ', '')))))
data_len_word = join['Consumer-complaint-summary'].apply(lambda x: len(str(x).split()))

Time Based Features

In [27]:
setc_date_month= join['Date-sent-to-company'].apply(lambda x:x.month)
setc_date_year= join['Date-sent-to-company'].apply(lambda x:x.year)
setc_date_day= join['Date-sent-to-company'].apply(lambda x:x.day)

In [28]:
dr_date_month= join['Date-received'].apply(lambda x:x.month)
dr_date_year= join['Date-received'].apply(lambda x:x.year)
dr_date_day= join['Date-received'].apply(lambda x:x.day)

In [29]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words='english',max_df=1.0, min_df=1, max_features=1000)
tf_feat = vectorizer.fit_transform(join['Consumer-complaint-summary'])

In [30]:
tf_feat.shape

(61809, 1000)

In [31]:
from scipy.sparse import csr_matrix
tf_idf_arr= tf_feat.toarray()

#Inputs for our Data

In [32]:
#Transaction
enc_trans_type_train=enc_trans_type[0:43266]
enc_trans_type_test=enc_trans_type[43266:]

In [33]:
#Complaint reason
enc_complain_reason_train=enc_complain_reason[0:43266]
enc_complain_reason_test=enc_complain_reason[43266:]

In [34]:
enc_comp_response_train=enc_comp_response[0:43266]
enc_comp_response_test= enc_comp_response[43266:]

In [35]:
enc_cons_dispute_train=enc_cons_dispute[0:43266]
enc_cons_dispute_test= enc_cons_dispute[43266:]

In [36]:
enc_time_train=enc_time[0:43266]
enc_time_test=enc_time[43266:]

In [37]:
tf_idf_arr_train=tf_idf_arr[0:43266]
tf_idf_arr_test=tf_idf_arr[43266:]

In [38]:
data_len_train = data_len[0:43266]
data_len_test = data_len[43266:]

In [39]:
data_len_char_train = data_len_char[0:43266]
data_len_char_test = data_len_char[43266:]

In [40]:
data_len_word_train = data_len_word[0:43266]
data_len_word_test  = data_len_word[43266:]

In [41]:
setc_date_month_train=setc_date_month[0:43266]
setc_date_month_test=setc_date_month[43266:]

In [42]:
#setc_date_year_train = setc_date_year[0:43266]
#setc_date_year_test = setc_date_year[43266:]

In [43]:
setc_date_day_train = setc_date_day[0:43266]
setc_date_day_test = setc_date_day[43266:]

In [44]:
dr_date_month_train = dr_date_month[0:43266]
dr_date_month_test = dr_date_month[43266:]

In [45]:
#dr_date_year_train = dr_date_year[0:43266]
#dr_date_year_test = dr_date_year[43266:]

In [46]:
dr_date_day_train = dr_date_day[0:43266]
dr_date_day_test = dr_date_day[43266:]

In [47]:
#Random Forest

In [63]:
train_x = np.hstack((enc_trans_type_train.reshape(-1,1) ,
                  enc_complain_reason_train.reshape(-1,1),
                  enc_comp_response_train.reshape(-1,1),
                  enc_cons_dispute_train.reshape(-1,1),
                  enc_time_train.reshape(-1,1),
                  tf_idf_arr_train,
                  np.array(data_len_train).reshape(-1,1),
                  np.array(data_len_char_train).reshape(-1,1),
                  np.array(data_len_word_train).reshape(-1,1),
                  np.array(setc_date_month_train).reshape(-1,1),
                  np.array(setc_date_day_train).reshape(-1,1),
                  np.array(dr_date_month_train).reshape(-1,1),
                  np.array(dr_date_day_train).reshape(-1,1))) 

In [64]:
test_x = np.hstack((enc_trans_type_test.reshape(-1,1) ,
                   enc_complain_reason_test.reshape(-1,1),
                  enc_comp_response_test.reshape(-1,1),
                  enc_cons_dispute_test.reshape(-1,1),
                  enc_time_test.reshape(-1,1), 
                  tf_idf_arr_test,
                  np.array(data_len_test).reshape(-1,1),
                  np.array(data_len_char_test).reshape(-1,1),
                  np.array(data_len_word_test).reshape(-1,1),
                  np.array(setc_date_month_test).reshape(-1,1),
                  np.array(setc_date_day_test).reshape(-1,1),
                  np.array(dr_date_month_test).reshape(-1,1),
                  np.array(dr_date_day_test).reshape(-1,1)))

### XG BOOST

In [65]:
from xgboost import XGBClassifier

In [66]:
xgb_model = XGBClassifier(
#learning_rate = 0.02,
n_estimators= 2000,
max_depth= 30,
min_child_weight= 2,
learning_rate=0.01,
gamma=0.9,                        
subsample=0.9,
colsample_bytree=0.8,
objective= 'multi:softprob',
nthread= -1,
scale_pos_weight=1)

In [67]:
xgb_model.fit(train_x, y_cat)

KeyboardInterrupt: 

In [58]:
xgb_model.score(train_x, y_cat)

0.48717237553737347

In [52]:
print recall_score(y_cat, xgb_model.predict(train_x),average=None)
print precision_score(y_cat, xgb_model.predict(train_x),average=None)
print f1_score(y_cat, xgb_model.predict(train_x), average= None)

[0.19901112 0.48093294 0.71611072 0.44798725 0.47975078]
[0.05758226 0.86606815 0.25524918 0.2089802  0.0557971 ]
[0.08932039 0.61844152 0.37635211 0.28500792 0.09996754]


In [65]:
y_classes = xgb_model.predict(test_x)

In [66]:
y_pred=list(le5.inverse_transform(y_classes))

/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [67]:
submission=pd.DataFrame()
submission['Complaint-ID']=test['Complaint-ID']
submission['Complaint-Status']=y_pred

In [68]:
submission.to_csv(path_or_buf="/Users/s0c02nj/Desktop/Submission15_sub.csv", encoding='utf-8',index=False)

In [57]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5)
skf.get_n_splits(train_x, y_cat)

5

In [69]:
pred_rf=[]
for train_index, val_index in skf.split(train_x, y_cat):
    
    
    X_train, X_val = train_x[train_index], train_x[val_index]
    y_train, y_val = y_cat[train_index], y_cat[val_index]
    
    xgb_model.fit(X_train,y_train)
    
    print f1_score(y_val, xgb_model.predict(X_val), average= None)
    
    
    pred_rf.append(xgb_model.predict_proba(test_x))
   

[0.01204819 0.88413875 0.14977974 0.11179039 0.02985075]
[0.01197605 0.88284519 0.13230769 0.08672566 0.        ]


/Users/s0c02nj/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


[0.03614458 0.88337371 0.13353566 0.10452962 0.        ]
[0.04819277 0.88327332 0.15269461 0.09557522 0.03076923]
[0.01219512 0.88488973 0.15315315 0.1011535  0.        ]
